#### Importing all libraries

In [22]:
import os
import re
import spacy
import srsly
import typer
import random
import warnings
import string
import PyPDF2
import csv
import glob
import docx2txt
import pandas as pd
#import PharseMatcher
#import configparserparser
from spacy.matcher import PhraseMatcher
from spacy.training import Example
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy.training import docs_to_json
from spacy.util import minibatch, compounding
from spacy.matcher import PhraseMatcher
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")  

from datetime import datetime
import datefinder
from dateutil.parser import parse

from common import create_patterns
from common import create_patterns_org
from common import create_patterns_name
from common import create_patterns_email
from common import create_patterns_mobnum
from common import create_patterns_DOB

### ALL GetAll_docx and pdf from home folder

In [23]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
#import Pypdf
import glob
import re
import itertools # Used to merge list of lists


## Author: Deepinder
## Description: getallfilenames returns all files as a list in the specified path, also returns all files from subfolders 
##              if the getall parameter is passed as "Y"
## How to Call: print(getallfilenames(path="/home", getall='Y'))
## Comments: 1) FIXED: The method was looping multiple times in the subfolders is fixed, So i am not expecting any duplicates
##           2) FIXED: Changed the code to have local variable fn instead of having it declared globally.


def getallfilenames(path = 'home', getall = 'N'): # Definition of the Method
    fn = []
    if (path=='home'): # Get CURRENT DEFAULT PATH
        path = os.getcwd()    # Get current Directory
    elif (os.path.exists(path)==False): #Check if the passed path exists or exit with return.
        print ('Path Not found')
        return
    fn.append(glob.glob(path+'/*.doc*'))
    fn.append(glob.glob(path+'/*.pdf'))
    if getall == 'Y':
        for roots,dirs,files in os.walk(path): # This code is to find the directories
            if (roots!=path) and (path+"/." not in roots) and (len(files)>0): # Excluding any folders which starts with .(dot)
#                print("SS"+path,roots,len(dirs),len(files)) # This code is to find the directories
                fn.append(glob.glob(roots+'/*.doc*'))
                fn.append(glob.glob(roots+'/*.pdf'))
    
# Below code is to merge all list of lists into one list and remove blank list
    filenames = list(filter(None, fn)) ## Remove blank list 
    filenames = list(itertools.chain.from_iterable(filenames)) ## Remove list of list
#    filenames = list(set(filenames)) ## remove Duplicates ## Removed this because i am not expecting any duplicates
    return (filenames)

#### Method convertfileintotext

In [24]:
# Author: Sahil
#Description: convertfile into text returns text in the docx or pdf file, It extracts the file name from the path.
#             Paramter pass is the filepath 
# How to call:(convertfileintotxt(filepath))
# Comments: 1) It moves to a function to extract docs and return txt, same thing goes if we fohdn a pdf file 
def convertfileintotxt(filepath):
    filename = os.path.basename(filepath) # it will pick up the filename from the end of the path.
    print(filename)
    if filename.endswith(".docx"): #if filename ends with docx 
        ret_text=Extract_docx_file(filepath) #it will give call to extract docxfile function 
        return ret_text
    elif filename.endswith(".pdf"): #if filename ends with pdf 
        Extract_pdf_file(filename)#it will give call to extract pdf_file function
        ret_text=Extract_pdf_file(filepath)
        return ret_text
    else:
         print("Not a proper format file")  #or else it will print not a proper format.

### Methid extract docxfile

In [25]:
#  function to open a docx file 
## Author: Sahil
## Description:Extract docx file takes the docx file and uses docx2txt to read the file in return file in a variable called resume 
## How to Call:this function is called in convert file to text from there it will be called 
##Comments : Used python dox2txt to read doxx file in python 
def  Extract_docx_file(file):
        resume = docx2txt.process(file) #docx2txt used to read docx file 
        return resume # return resume when docx file found 


### Method Extract_pdf_file

In [26]:
#  function to open a pdf file 
## Description:Extract pdf file method takes the pdf file and uses pdf.reader to read the file in return file in a variable called resume 
## How to Call:this function is called in convert file to text from there it will be called 
##Comments : Used python pdffilereader to read pdf file in python 

def Extract_pdf_file(file):
    resume=""
    output = []
    
    if (os.path.isfile(file)==True):
        pdf_file = open(file,"rb")
        pdfReader = PyPDF2.PdfFileReader(pdf_file)
        count= pdfReader.numPages
        for i in range(count):
            page = pdfReader.getPage(i)
            output.append(page.extractText())
        for element in output:
            resume = resume + str(element )
        print("\nSuccessfully Read")
        return resume

#### Method Removecharacpunc

In [27]:
#function to remove characters such as , : - + spaces  
## Author: Sahil
## Description:Removecharpunc removes all the punctions from the resume and returns txtdata in remchrpunc
## How to Call: removecharacpunc(textdata)
## Comments: The method is remove all pipes , and all other things from the textdata 

def removecharacpunc(textdata):
    punc = '''!()[]{}();:'",<>?#$%^&*_~'''
    res = re.sub(' +', ' ', textdata)
    res= re.sub(', ', ' ', res)
    res=re.sub("’","",res)
#     res = res.replace("'", "")
#     res = res.replace("(", "")
#     res = res.replace(")", "")
    res = res.replace('|', " ")
    for ele in res:
        if ele in punc:
             remchrpunc = res.replace(ele, "")
    return remchrpunc


### Method removetabsnemptylines

In [28]:
# function to remove tab and extra new line
def removetabsnemptylines(remchar):
    
    lines = remchar.split("\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]

    string_without_empty_lines = ""
    for line in non_empty_lines:
          string_without_empty_lines += line + "\n"
    res=string_without_empty_lines.replace('\t', '')
    return res
    

### 7 Method Remove stop words

In [29]:
#Author:Sahil
#Description: removing all the stop words from the doc objec....We would be using spacy attriubte nto remove stopwords i.e token.is_stop
#How to call a function: removestopwords(doc)---where doc is the input string 
#Comments: Hear I have removed all the stop words and also got the lenth of my doc object and my cleaned object.



def removestopwords(doc):
    stopwrd_str = ""  # making a string 
    my_doc_cleaned = [token for token in doc if not token.is_stop and not token.is_punct]
    for token in my_doc_cleaned:
        stopwrd_str=stopwrd_str + ' ' + token.text    
    return stopwrd_str

### 8.Method replace lemma.

In [30]:
# Author: Sahil 
# Description:Replace all the words with the lemma gives the output into a list
##How to call: replacelemma(doc,res)---> where doc is a doc object and res is a string.
##Comments: 1. I have made the fuction by using a temporary doc object while in real we would be passing resume doc object.
##          2. I have used the replace command to replace one string to onother and then store it in temp  
## Coomment: Hear lemmatization is done as playing becomes play From becomes from.



#res = ("I am playing football and will be going to school From Playing tommorow")
#print(type(res))
#doc = nlp(res)
#print(type(doc))
def replacelemma(doc,res):
 
    for token in doc:
        ntoken = str(token)
        nlemma = str(token.lemma_)
        if ntoken != nlemma:
            #print(ntoken + ' ' + nlemma)
            temp=res.replace(ntoken, nlemma)
            res=temp
    return res

## 9. Method read company using csv

In [31]:
# Author: Sahil 
# Description:Get all the company name from the csv df and put it into a list and running a for loop to match it with input string and
##            gives the output into a list
##How to call: readskills(doc)---> where doc is input string 
##Comments: 1. I have made the fuction by using a temporary doc object while in real we would be passing resume doc object.
##          2. I have to add more cpmapnay names to the Csv to it should not miss any of it
## Coomment: Other method of extracting company names are pattern way  

#Right
nlp=spacy.load("en_core_web_sm")
#doc2=nlp("Aaditya techonologies and Impluse software solutions are two big companies with the help of wal mart company ")
def readcompanyname(doc):
   
    matcher = PhraseMatcher(nlp.vocab)
    tokens = [token.text for token in doc]
    df = pd.read_csv("Maincsvv.csv")

    lst= [] 
    for i in df["Company_Name"]:
        lst.append(i)     
    #print(lst)

    #obtain doc object for each word in the list and store it in a list
    
    patterns = [nlp(cn) for cn in lst]
    #add the pattern to the matcher
    matcher.add("compnay_name", patterns)
    matches = matcher(doc)

    cnlist = []
    for match_id, start, end in matches:
        span = doc[start:end]
        cnlist.append(span.text)
    return cnlist
    complist = list(dict.fromkeys(cnlist))
   # print(aalist)
    return complist


#### 10.Method readskills 

In [32]:
### # Author: Sahil 
# Description:Get all the skills from the csv df and put it into a list and running a for loop to match it with input string and
##            gives the output into a list
##How to call: readskills(doc)---> where doc is input string 
##Comments: 1. I have made the fuction by using a temporary doc object while in real we would be passing resume doc object.
##          2. I have to add more skills to the Csv to it should not miss any of it
## COmment 1. Other method to ffind skills is make patterns of all the skills in the common.py and then find it with matcher]


#doc = nlp("This is the biggest Java programming core php seminar, Java is good in  Sql Server I have come Core Java hear HTML5 to learn CSS MySQL")
def readskills(doc):
    matcher = PhraseMatcher(nlp.vocab) #PhraseMatcher lets you efficiently match large terminology lists.
    tokens = [token.text for token in doc] # Iterating through the loop to store token.text in a list
   
    df = pd.read_csv("Maincsvv.csv")      # reading data from a csv file and storing it in dataframe             
    lst= []                               # making a list to store the skills
    for i in df["Skills"]: # for data from  dataframe column 'Skills'
        lst.append(i)# appending all column values to a list 
    patterns = [nlp(cn) for cn in lst] 
    matcher.add("Skills",patterns) #adding all the skills to the pattern
    matches = matcher(doc)
    
    cnlist = []
    for match_id, start, end in matches:
        span = doc[start:end]
        cnlist.append(span.text)
    skilllist = list(dict.fromkeys(cnlist))
    return skilllist

#### ## 11. Method to Extract email.


In [33]:

##Author: Sahil 
##Description:Get all the email address from the inputstring i.e doc object.
##How to call: extractemail(doc)--->doc is the input paramter.
#Comments: 1. It will extract all the emails address from the inputstring and store it in a list and return the list  
##         2. Right now we are not taking the whole resume doc object but passing a temporary doc object which has emails in it 
##            to check if it is working.




#doc = nlp("sahil@yahoo.com yhyhy as.Talend@gmail.com 813-204-0866 | sahil_hodekar.Sahil@gmail.com IND123@gmail.com service@yahoo.in sahil@offsoar.com")#Making a nlp doc object
# # function to extract email
def extractemail(doc):      # takes doc object as input parameter.
    email_lst = []           # making a list to store the email address
    for token in doc:       # Iterate the loop for all the tokens in doc 
        if token.like_email: # Matching the tokens which are email through `like_email` attribute
            email_lst.append(token.text)  # append the matched tokens to a list
    return email_lst         # returns list.

#### #12.Method mobile Number.

In [34]:

# Author: Sahil 
# Description: Get the mobile number from the input string i.e doc object.
##How to call: extractnumbers(doc)--->doc is the input paramter.
#Comments: 1. It will extract all the numbers from the inputstring and store it in a list and return the list  
##         2. Right now we are not taking the whole resume doc object but passing a temporary doc object which has mobile numbers in it 
##            to check if it is working
##         3. Problems faced using like.num spacy atttribute is IT does not extract all the numbers like for eg. Numbers in 348-345-433 and (345)434
##         4.It picks numbers like +919967011417 9987177046
## Solution  -- I have to make patterns using regex to extract mobile numbers. 

nlp = spacy.load("en_core_web_sm")#Loading a default spacy model 
#doc = nlp("(I am having (456)234567 mobile 348-234-345 number 99387177046 But earlier +3439439393 my mobile 41234344333 number was +919967011417")#Making a nlp doc object 
def extractmobnumbers(doc): #putting the input as doc object.
    num_lst=[] #making a list to store the number 
    for token in doc: #For all the tokens in doc it will take a for loop 
        if token.like_num: #using spacy default .like_num it finds the number 
            num = (token)
            if(len(num) >=10):#Checking if the length of the token is more than 10 digits 
                num_lst.append(num)#if the number is more than 10 digit or 10 digit add it to the numlist
    
    phno=phoneno(doc)
    num_lst.append(phno)
    return num_lst#returns numlist at then end.


In [45]:
res = Extract_docx_file('7499495.docx')
#print(res)
extractname(res)

['Phone: +14157876954']

In [43]:
extractname(res)

['Phone: +14157876954']

In [42]:
res = convertfileintotxt('C:\Users\sahil\Nova31082021\7499495.docx')
print(res)
extractname(res)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-42-376ed7f6a5f3>, line 1)

### 13 Extract first name last name 

In [35]:
# Author: Sahil
## Description: This function helps us extracting candidate name from the resume 
## How to Call: (extractname(res) ---res string to be passed as input paramter.
## Comments: 1) Functions extract first line of the String and if first line is the "RESUME" or CURRICULUM VITAE 
##                it extract second line and extract candidate name from the second line.

def extractname(res): 
    listname=[]
    candidatenname = res.split('\n', 1)[0] # this code is use to extract the first line from the string
    if candidatenname == "RESUME" or candidatenname == "CURRICULUM-VITAE" or candidatenname =="CURRICULUM VITAE":# checking if the first line is resume or cirriculm vitae
        candidatenname = res.split('\n',2)[1] # # this code is use to extract the second line from the string
        candidatenname = candidatenname.strip()# it removes starting and end spaces 
        if candidatenname =="Name":
            candidatenname = res.split('\n',3)[0] #this code is use to extract the third line from the string
    listname.append(candidatenname)
    return listname
extractname(res)

['s']

## 24. Read Qualifications

In [15]:
# Author: Sahil 
# Description:Get all the qualification into a list and make  a for loop to match it with input string and
##            gives the output into a list
##How to call: qualtifications(doc)---> where doc is input string 
##Comments: 1. I have made the fuction by using a temporary doc object while in real we would be passing resume doc object.
##          2. I have to add more degree names to the list should not miss any of it
  

#doc = nlp("I enjoy B.S.C working in Purplejack M.C.A more than I used to be in BSC (COMP SCI.) . To work in Apple is my Great ")
def extract_qualification(doc):
    
    matcher = PhraseMatcher(nlp.vocab)

    Qualification = ['B.S.C' , 'B.A','M.C.A','Bachelor of Engineering','Higher School Certificate','Secondary School Certificate' ,
             'BSC (COMP SCI.)','BCA','B. Tech','Bachelor in technology','Bachelor of Science (BS)',
            'M.Sc. in Computers','M.Sc.','M. Tech','MCA','Masters in Computer Applications','Master in Systems Management',
            'M.S',' Bachelor in Computer Applications','B.C.A','Bachelor in Computer Science',
            'Masters in Computer Science']

#obtain doc object for each word in the list and store it in a list
    patterns = [nlp(cn) for cn in Qualification] #making a doc object of all the words.
#add the pattern to the matcher
    matcher.add("Qualification_pattern", patterns) 
    matches = matcher(doc)
    qualification_list = []
    for match_id, start, end in matches:
        span = doc[start:end]
        qualification_list.append(span.text)
    return qualification_list 

### DATE PATTERN

In [16]:
## Author: Sahil
## Description: Extract dates method has used patterns to take out all dates from the passed doc file 
## How to Call: extractdates(doc)
## Comments: 1) FIXED: The method was picking out anything that comes before digit like sahil26 or c-11, so changed patterns 
##           2) FIXED: as after making patterns the patterns wer repating so used dict.fromkeys to remove repated words 
##           3) FIXED: Then  IT MATCHES IF found date is Jan 97 then it will change to Jan1997 or if the date is Jan18 it will print Jan2018
def extractdates(doc):
    matcher = Matcher(nlp.vocab)  
    date_pattern1 = [{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|Dec(?:ember)$'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_1', [date_pattern1])
        
    date_pattern2 = [{'TEXT':{'REGEX':r'^\d{2}$'}},{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_2', [date_pattern2])

    date_pattern3 = [{'TEXT':{'REGEX':r'^\d{1}$'}},{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_3', [date_pattern3])

    date_pattern4 = [{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sep?(?:tember)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'}},{'TEXT':{'REGEX':r'^\d{2}$'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_4', [date_pattern4])

    date_pattern5 = [{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'}},{'TEXT':{'REGEX':r'^\d{2}(?:st|nd|rd|th)?$'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_5', [date_pattern5])
    
    date_pattern6 = [{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'}},{'TEXT':{'REGEX':r'^\d{1}(?:st|nd|rd|th)?$'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_6', [date_pattern6])
 

    date_pattern8 = [{'TEXT':{'REGEX':r'^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$'}}]
    matcher.add('DATE_PATTERN_8', [date_pattern8])  
    
    date_pattern11 = [{'TEXT':{'REGEX':r'^\d{1,2}?[/:\']\d{2,4}$'}}] # eg 02/2021 8/2021
    matcher.add('DATE_PATTERN_8', [date_pattern11])

    date_pattern9 = [{'TEXT':{'REGEX':r'([0]?\d|[1][0-2])[/-]([0-3]?\d)[/-]([1-2]\d{3}|\d{2})'}}]
    matcher.add('DATE_PATTERN_9', [date_pattern9])
    
    
    date_pattern11 = [{'TEXT':{'REGEX':r'^\d{2}(?:st|nd|rd|th)?$'}},{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?$'}},{'TEXT':{'REGEX':r'^\d{2,4}$'}}]
    matcher.add('DATE_PATTERN_11', [date_pattern11])
    
#     date_pattern12 = [{'TEXT':{'REGEX':r'^Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May?|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|Dec(?:ember)'}}]
#     matcher.add('DATE_PATTERN_12', [date_pattern12])
    

#   date_pattern10 = [{'IS_DIGIT': True},
#            {'IS_PUNCT': True},
#            {'IS_DIGIT': True},
#            {'IS_PUNCT': True},
#            {'IS_DIGIT': True}]

#     matcher.add('DATE_PATTERN_10', [date_pattern10])  
    
    
    lstdate=[] 
    matches = matcher(doc)
    
    def string_set(datelist):
         return set(i for i in datelist 
                if not any(i in s for s in datelist if i != s))

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        lstdate.append(span.text)
    datelist = list(dict.fromkeys(lstdate))
    datelist1=string_set(datelist)
    
    def string_set(datelist):
         return set(i for i in datelist 
                if not any(i in s for s in datelist if i != s))

    str1="" 
    lsts=[]
    for ele in datelist1:
        if (re.match("(Jan)([0-9]{2})",ele) or re.match("^(January[\d][\d])",ele)
            or re.match("(Feb)([0-9]{2})",ele) or re.match("(^February[\d\d])",ele)
            or re.match("(Mar)([0-9]{2})",ele) or re.match("(^March[\d\d])",ele)
            or re.match("(Apr)([0-9]{2})",ele) or re.match("(^April[\d\d])",ele)  
            or re.match("(May)([0-9]{2})",ele) or re.match("(^June[\d\d])",ele)
            or re.match("(July)([0-9]{2})",ele) or re.match("(^Aug[\d\d])",ele)
            or re.match("(August)([0-9]{2})",ele) or re.match("(^Sep[\d\d])",ele)
            or re.match("(Sept)([0-9]{2})",ele) or re.match("(^September[\d\d])",ele)
            or re.match("(Oct)([0-9]{2})",ele) or re.match("(^October[\d\d])",ele)
            or re.match("(Nov)([0-9]{2})",ele) or re.match("(^November[\d\d])",ele)
            or re.match("(Dec)([0-9]{2})",ele) or re.match("(^December[\d\d])",ele)
           ): 
            lsts.append(ele)
    
    for ele in lsts:
        datetwodigit=ele[-2:]
        if int(datetwodigit) > 50: 
            ele = ele[:-2] + "19" + str(datetwodigit)
        elif int(datetwodigit)<50:
            ele = ele[:-2] + "20" + str(datetwodigit)
    
        
    lstappend=[]
    for ele in lsts:
        datetwodigit=ele[-2:]
        if int(datetwodigit) > 50: 
            ele = ele[:-2] + "19" + str(datetwodigit)
            lstappend.append(ele)
        elif int(datetwodigit)<50:
            ele = ele[:-2] + "20" + str(datetwodigit)
            lstappend.append(ele)
        
    remlst = list(set(datelist)^set(lsts))
     

    final_date_lst= list(remlst) + lstappend

    sorted_date_list=[]
    for date_string in final_date_lst:
        sorted_date_list.append(parse(date_string).strftime("%Y-%m-%d"))

    sorted_date_list.sort(key=lambda date: datetime.strptime(date, "%Y-%m-%d"))
    
    return(sorted_date_list)

In [17]:
# Author: Sahil 
## Description: adding all the patterns to th method and making a list of all the patterns 
## How to Call:load_RBM(res) 
## Comments:1) Made two  functions one consist of all patterns and then loading that patterns in laad_RBM
##           2) Made list of names proglang orgcomp email 
##           3) things that are extracted from pattern will be stored in a list. 
def make_RBM():
    print("Uploading Rule Based Model")
    nlp = spacy.load("en_core_web_sm")
    nlp.remove_pipe("ner")
    ruler = nlp.add_pipe("entity_ruler")
    # # add entity ruler
    # nlp.add_pipe("entity_ruler", before="ner")

    ruler.add_patterns([{'label':'PRROGlANG','pattern':p } for p in create_patterns()])
    ruler.add_patterns([{'label':'Org_Comp','pattern':po } for po in create_patterns_org()])
    ruler.add_patterns([{'label':'Name','pattern':nam } for nam in create_patterns_name()])
    ruler.add_patterns([{'label':'Email','pattern':em } for em in create_patterns_email()])
    ruler.add_patterns([{'label':'Mobile_No.','pattern':mn } for mn in create_patterns_mobnum()])
    ruler.add_patterns([{'label':'DOB.','pattern':mn } for mn in create_patterns_DOB()])
    ruler.to_disk("./patterns.jsonl")
    nlp.to_disk("./res-model")
    print("Rule based Model Done")

def load_RBM(res):
    nlp_resdoc = spacy.load("res-model")   
    doc1=nlp_resdoc(res)
    print("\n\n \t\tPRINTING DATA \n\n  ( RULE BASED MODEL)")
    names=[]
    programlang=[]
    Orgcomp=[]
    emails=[]
    contactno = []
    for ent in doc1.ents:
        if (ent.label_ == 'Name'):
            names.append(ent.text)
        if (ent.label_ == 'PRROGlANG'):
            programlang.append(ent.text)
        if (ent.label_ == 'Org_Comp'):
            Orgcomp.append(ent.text)
        if (ent.label_ == 'Email'):
            emails.append(ent.text)
        if (ent.label_ == 'Mobile_No.'):
            contactno.append(ent.text)
    print("NAMES:")
    for nm in names:
        print("\t  " , nm)
        
    print("SKILLS:")
    for pl in programlang:
        print("\t  " , pl)    
        
    print("Worked at ")
    for oc in Orgcomp:
        print("\t  " , oc)   
        
    print("Contact No. ")
    for cn in contactno:
        print("\t  " , cn)   
    print("Email for Correspondence ")
    for em in emails:
        print("\t  " , em)   
        


In [19]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import spacy
from spacy.matcher import Matcher


def phoneno(doc):
    nlp = spacy.load("en_core_web_sm")
    print("hiiiiii")
    matcher = Matcher(nlp.vocab)

    phone_reg = [{"TEXT": {"REGEX": "^[\+\(]?[0-9][0-9 .\-\\s(\)]{8,}[0-9]"}},]
    phone_reg1 = [{'ORTH': '('},{'SHAPE': 'ddd'},{'ORTH': ')'},{'SHAPE': 'dddd'},{'ORTH': '-', 'OP': '?'},{'SHAPE': 'dddd'}]
    phone_rex3 = [{"TEXT": {"REGEX": "^[\(\w]+\)\S+\-\d"}},]
    phone_reg4 = [{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'dddd'}]
    phone_reg5 = [{'ORTH': '('},{'SHAPE': 'ddd'},{'ORTH': ')'},{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'dddd'}]
    phone_rex2 = [{"TEXT": {"REGEX": "{^(\(?\d{3}\)?)},{ (\d{3}-\d\d\d\d)"}},]
    matcher.add("number", [phone_reg,phone_reg1,phone_rex2,phone_reg4,phone_reg4,phone_reg5])

 
    phonenolist=[]

    matches=matcher(doc)
    print()
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id] 
        span = doc[start:end]  # The matched span
        phonenolist.append(span.text)
        print(span.text)
    return phonenolist


In [20]:
def outputmethod():
    filenamepathlist= getallfilenames(path = 'home', getall = 'N')
    for i in range(len(filenamepathlist)):
        mainlist=[]
        filepath = filenamepathlist[i]
        converted_text_resume = convertfileintotxt(filepath)
        remo_char_punc = removecharacpunc(converted_text_resume)
        res_cleantxt = removetabsnemptylines(remo_char_punc)
#         print(res_cleantxt)
        res_doc = nlp(res_cleantxt)
        stopwrd_str = removestopwords(res_doc)
        resume = replacelemma(res_doc,stopwrd_str)
        companylist = readcompanyname(res_doc)
        
        Skilllist = readskills(res_doc)
        
        mobnumber = extractmobnumbers(res_doc)
        
        email = extractemail(res_doc)
       
        candidate_name =  extractname(res_cleantxt)
        qualifcationlist = extract_qualification(res_doc)
        
        date_list = extractdates(res_doc)
        
        
        rolelist= rolesdesignation(res_cleantxt)
        
#         phoneno(res_doc)
#         print("hello")
        import csv
        from itertools import zip_longest
        data = [candidate_name,rolelist,companylist, Skilllist,mobnumber,email,qualifcationlist,date_list]
        export_data = zip_longest(*data, fillvalue = '')
        with open('candidate13.csv', 'w', encoding="ISO-8859-1", newline='') as file:
            write = csv.writer(file)
            write.writerow(("Name","Role","Company", "Skills","mobnumber","email","qualifcationlist","Date"))
            write.writerows(export_data)

In [21]:
outputmethod()

Peter-Smith.pdf

Successfully Read

Successfully Read


UnboundLocalError: local variable 'remchrpunc' referenced before assignment

In [ ]:
# s = "this is a sentense how is the weather Is that going to work "
# target = "Role"
# words = s.split()
# listrole=[]
# for i,w in enumerate(words):
#     if w == target:
#         # next word
#         listrole.append(words[i+1])
#         listrole.append(words[i+2])
# for i in listrole:
#     print(i)

In [ ]:
outputmethod()

In [ ]:
s = "this is a sentense how is the weather Is that going to work "
target = "is"
words = s.split()
listrole=[]
for i,w in enumerate(words):
    if w == target:
        # next word
         listrole.append(words[i+1])
    if i>0:
        # previous wordif i>0:
        print(words[i-1])

## METHODS MADE BY REGEX WHERE POSSIBLE

In [ ]:
#REGEX METHOD TO FIND EMAIL.

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)


EMAIL_REG = [{"TEXT": {"REGEX": "^[\w+\.[a-zA-Z0-9]+@[\w]+\.[\w]+"}},]



matcher.add("Email", [EMAIL_REG])
doc=nlp("as.Talenil@yahoo.com yhyhy as.Talend@gmail.com | 813-204-0866 | IND123@gmail.com service@yahoo.in sahil@offsoar.com")
matches=matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id] 
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

phone_reg = [{"TEXT": {"REGEX": "^[\+\(]?[0-9][0-9 .\-\\s(\)]{8,}[0-9]"}},]
phone_reg1 = [{'ORTH': '('},{'SHAPE': 'ddd'},{'ORTH': ')'},{'SHAPE': 'dddd'},{'ORTH': '-', 'OP': '?'},{'SHAPE': 'dddd'}]
phone_reg1 = [{'ORTH': '('},{'SHAPE': 'ddd'},{'ORTH': ')'},{'SHAPE': 'ddd'},{'ORTH': '-', 'OP': '?'},{'SHAPE': 'dddd'}]
phone_rex2 = [{"TEXT": {"REGEX": "^[\(]\d{3}[\)]\d{3}-\d{4}"}},]
phone_rex3 = [{"TEXT": {"REGEX": "^[\(\w]+\)\S+\-\d"}},]
phone_reg4 = [{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'dddd'}]
phone_reg5 = [{'ORTH': '('},{'SHAPE': 'ddd'},{'ORTH': ')'},{'SHAPE': 'ddd'},{'ORTH': '-'},{'SHAPE': 'dddd'}]
matcher.add("number", [phone_reg,phone_reg1,phone_rex2,phone_reg4,phone_reg4])


doc2 = nlp("""9967011417
(415)312-1234
+919967011417
(914) 368-2698
(967)981-9827 _______
+91(976)006-4000
020 7183 8750
813-204-0866
+14155552671
(123) 4567 8901
+447890123456
(408) 960-5241 
‪
"""
)


matches=matcher(res_doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id] 
    span = doc2[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)


In [ ]:
pattern = [{'ORTH': '('},
           {'SHAPE': 'ddd'},
           {'ORTH': ')'},
           {'SHAPE': 'dddd'},
           {'ORTH': '-', 'OP': '?'},
           {'SHAPE': 'dddd'}]

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("phonenumber", [pattern])

In [ ]:
#doc = nlp("call me at (123) 5496 7820")

In [ ]:
matches = matcher(doc)
matches

In [ ]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id] 
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

###### 

In [ ]:
def call_output()
